# 쿠팡 '노트북' 검색 후 데이터 스크래핑

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

## 참고자료

1. get 방식
- https://www.coupang.com/np/search?q=노트북&channel=user&component=&eventCategory= ..... 
- 위와 같이 /np/search? ~~~~ 값이 나옴
- ?뒤 부터 변수와 값을 의미
- 변수와 값은 &로 표시
- 쉽게 페이지를 요청할 수 있음
- 데이터 전송에 제약이 존재하여 큰 데이터는 보낼 수 없음


2. post 방식
- https body에 숨겨서 보내는 방식
- 보안데이터는 post방식으로 숨겨서 전송하는 방식이 안전(개인정보, 회원가입 등)
- 데이터 전송에 제약이 없음

# 1. BeautifulSoup로 스크래핑

In [2]:
url = 'https://www.coupang.com/np/search?q=%EB%85%B8%ED%8A%B8%EB%B6%81&channel=user&component=&eventCategory=SRP&trcid=&traid=&sorter=scoreDesc&minPrice=&maxPrice=&priceRange=&filterType=&listSize=36&filter=&isPriceRange=false&brand=&offerCondition=&rating=0&page=1&rocketAll=false&searchIndexingToken=1=5&backgroundColor='
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'}
res = requests.get(url, headers = headers)
res.status_code
soup = BeautifulSoup(res.text, 'lxml')

# 2. soup.find_all 함수로 조건에 맞는 값을 items라는 매개변수로 생성

- 여기서 re.compile('^search-product')을 사용한 이유는 광고가 붙지 않은 것은 search-product / 광고가 붙은 것은 search-product search-product__ad-badge이기 때문

In [3]:
items = soup.find_all('li', attrs={'class':re.compile("^search-product")})
items[0].find('div', attrs={'class':'name'}).get_text()

'삼성 게이밍북 NT371B5J I5-4310M/8G/SSD256G/G820M/15.6/WIN10, WIN10 Pro, 8GB, 256GB, 코어i5, 블랙'

# 3. items 변수 중 조건에 맞는 값 출력

### 1) 전체 출력

In [23]:
li_name =[]
li_price = []
li_rate =[]
li_rate_count = []
notebook = pd.DataFrame(columns = ['name','price','rate','rate_count'])

for i in items:
    name = i.find('div', attrs={'class':'name'}).get_text()                       # 제품명
    price = i.find('strong', attrs={'class':"price-value"}).get_text()            # 가격
    rate = i.find('em', attrs={'class':"rating"})                                 # 평점
    if rate:                                                                      # em 태그에 텍스트가 없는것(평점이 없음)이 존재
        rate = rate.get_text()                                                    # 따라서 if문으로 분리
    rate_count = i.find('span', attrs={'class':"rating-total-count"})             # 평점 수
    if rate_count:
        rate_count =rate_count.get_text()
    li_name.append(name)
    li_price.append(price)
    li_rate.append(rate)
    li_rate_count.append(rate_count)
            
li_notebook =[li_name, li_price, li_rate, li_rate_count]
for i in range(0,4):
    notebook.iloc[:,i]= li_notebook[i]
    
display(notebook)

,name,price,rate,rate_count
0,"에이수스 2022 노트북 17.3, Mecha Gray, ASUS TUF Gamin...","1,749,000",None,None
1,"삼성전자 갤럭시북 프로 15.6, 실버, NT950XDC-X71A, 코어i7, 1T...","1,749,000",None,None
2,"에이수스 2022 노트북 17.3, Mecha Gray, ASUS TUF Gamin...","1,549,000",None,None
3,"LG전자 2020 울트라 PC 14, 화이트, 셀러론, 512GB, 8GB, WIN...","526,000",4.5,(415)
4,"삼성전자 갤럭시북 프로360 15.6, 미스틱 실버, 코어i5, 1TB, 16GB,...","1,649,000",5.0,(263)
5,이태원클라쓰북 그램스타일 노트북 풀패키지미개봉 NB141LTN41 8세대 14 IP...,"299,000",4.5,(556)
6,LG전자 2021 울트라 PC 15.6 + HDMI케이블 + 무선마우스 + 마우스패...,"799,000",5.0,(403)
7,"베이직스 2021 베이직북13 2세대, 베이직 골드, BB1321FW, 셀러론, 2...","361,770",4.5,(54)
8,"삼성전자 노트북 플러스2 15.6, 블레이드 블랙, NT560XDZ-G78AB, 코...","1,449,000",4.5,(256)
9,"레노버 2021 노트북 15.6, Iron Grey, 라이젠5, 256GB, 8GB...","449,000",4.5,(15)


### 2) 광고 제외 출력

In [29]:
li_name =[]
li_price = []
li_rate =[]
li_rate_count = []
notebook = pd.DataFrame(columns = ['name','price','rate','rate_count'])

for i in items:
    ad_badge = i.find('span', attrs = {'class' : 'ad-badge-text'})
    if not ad_badge:
        name = i.find('div', attrs={'class':'name'}).get_text()                     
        price = i.find('strong', attrs={'class':"price-value"}).get_text()         
        rate = i.find('em', attrs={'class':"rating"})                              
        if rate:                                                                   
            rate = rate.get_text()                                                   
        rate_count = i.find('span', attrs={'class':"rating-total-count"})             
        if rate_count:
            rate_count =rate_count.get_text()
        li_name.append(name)
        li_price.append(price)
        li_rate.append(rate)
        li_rate_count.append(rate_count)

li_notebook =[li_name, li_price, li_rate, li_rate_count]
for i in range(0,4):
    notebook.iloc[:,i]= li_notebook[i] 

display(notebook)

,name,price,rate,rate_count
0,"에이수스 2022 노트북 17.3, Mecha Gray, ASUS TUF Gamin...","1,749,000",None,None
1,"에이수스 2022 노트북 17.3, Mecha Gray, ASUS TUF Gamin...","1,549,000",None,None
2,"LG전자 2020 울트라 PC 14, 화이트, 셀러론, 512GB, 8GB, WIN...","526,000",4.5,(415)
3,LG전자 2021 울트라 PC 15.6 + HDMI케이블 + 무선마우스 + 마우스패...,"799,000",5.0,(403)
4,"베이직스 2021 베이직북13 2세대, 베이직 골드, BB1321FW, 셀러론, 2...","361,770",4.5,(54)
5,"레노버 2021 노트북 15.6, Iron Grey, 라이젠5, 256GB, 8GB...","449,000",4.5,(15)
6,"HP 2021 노트북 15s, NVMe 256GB, 윈도우 포함, 8GB","779,000",5.0,(82)
7,"레노버 2021 노트북 V15, Iron Gray, 라이젠5, 256GB, 8GB,...","599,000",4.5,(15)
8,\n ...,"1,598,000",5.0,(642)
9,\n ...,"1,448,840",5.0,(8159)


### 3) 리뷰 100개 이상, 평점 4.5 이상 되는 것만 조회

In [52]:
li_name =[]
li_price = []
li_rate =[]
li_rate_count = []
notebook = pd.DataFrame(columns = ['name','price','rate','rate_count'])

for i in items:
    ad_badge = i.find('span', attrs = {'class' : 'ad-badge-text'})
    if not ad_badge:
        name = i.find('div', attrs={'class':'name'}).get_text()                       
        price = i.find('strong', attrs={'class':"price-value"}).get_text()

        rate = i.find('em', attrs={'class':"rating"})                                
        if rate:                                                                      
            rate = rate.get_text()
    
        rate_count = i.find('span', attrs={'class':"rating-total-count"})
        if rate_count:
            rate_count = rate_count.get_text()
            rate_count = rate_count[1:-1]            # 평점 출력 시 괄호를 제거해주기 위해
        if float(rate) >= 4.5 and int(rate_count) >=100 :
            li_name.append(name)

TypeError: float() argument must be a string or a number, not 'NoneType'

In [49]:
float(rate)

5.0

In [55]:
li_name =[]
li_price = []
li_rate =[]
li_rate_count = []
notebook = pd.DataFrame(columns = ['name','price','rate','rate_count'])

for i in items:
    ad_badge = i.find('span', attrs = {'class' : 'ad-badge-text'})
    if not ad_badge:
        name = i.find('div', attrs={'class':'name'}).get_text()                       
        price = i.find('strong', attrs={'class':"price-value"}).get_text()

        rate = i.find('em', attrs={'class':"rating"})                                
        if rate:                                                                      
            rate = rate.get_text()
        else:
            continue
    
        rate_count = i.find('span', attrs={'class':"rating-total-count"})
        if rate_count:
            rate_count = rate_count.get_text()
            rate_count = rate_count[1:-1]            # 평점 출력 시 괄호를 제거해주기 위해
        else:
            continue
        
        # 리뷰 100개 이상, 평점 4.5 이상 출력
        if float(rate) >= 4.5 and int(rate_count) >=100 :
            li_name.append(name)
            li_price.append(price)
            li_rate.append(rate)
            li_rate_count.append(rate_count)
            
li_notebook =[li_name, li_price, li_rate, li_rate_count]
for i in range(0,4):
    notebook.iloc[:,i]= li_notebook[i]
    
display(notebook)

,name,price,rate,rate_count
0,"MSI 2021 SWORD GF76 17.3, 블랙, 코어i7 11세대, 512GB...","1,497,300",5.0,231
1,"한성컴퓨터 2020 TFX 15.6, 라이젠7 3세대, 500GB, 16GB, WI...","1,160,000",5.0,163
2,"레노버 2021 ThinkPad E15, 블랙, 라이젠7 4세대, 256GB, 8G...","801,570",5.0,408
3,"삼성전자 2019 갤럭시북 이온 15.6, 아우라 실버, 코어i5 10세대, 256...","1,446,000",5.0,482
4,"레노버 2021 IdeaPad Slim3 15.6, SAND, 코어i3 11세대, ...","559,700",5.0,179
5,"레노버 2021 IdeaPad Slim3 15.6, 골드, 펜티엄, 128GB, 4...","422,200",5.0,179
6,"레노버 2021 IdeaPad Slim3 15.6, SAND, 코어i3 11세대, ...","998,000",5.0,179
7,"MSI 2021 SWORD GF76 17.3, 화이트, 코어i7 11세대, 512G...","1,497,300",5.0,231
8,"Apple 2020 맥북 에어 13, 골드, M1, 256GB, 16GB, MAC ...","1,448,840",5.0,8160
9,"주연테크 2020 캐리북T 13.3, 메탈, 펜티엄, 128GB, 4GB, WIN1...","448,830",4.5,171


### 4) 1~5p에서 광고가 아니며, 리뷰 100개 이상, 평점 4.5 이상 되는 것만 조회

In [53]:
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'}
li_page = []
li_name =[]
li_price = []
li_rate =[]
li_rate_count = []
li_link = []
notebook = pd.DataFrame(columns = ['page','name','price','rate','rate_count','link'])

for i in range(1,6):
    url = 'https://www.coupang.com/np/search?q=%EB%85%B8%ED%8A%B8%EB%B6%81&channel=user&component=&eventCategory=SRP&trcid=&traid=&sorter=scoreDesc&minPrice=&maxPrice=&priceRange=&filterType=&listSize=36&filter=&isPriceRange=false&brand=&offerCondition=&rating=0&page={}&rocketAll=false&searchIndexingToken=1=5&backgroundColor='.format(i)
    res = requests.get(url, headers = headers)
    soup = BeautifulSoup(res.text, 'lxml')
    items = soup.find_all('li', attrs = {'class': re.compile('^search-product')})

    for item in items:
        ad_badge = item.find('span', attrs = {'class' : 'ad-badge-text'})
        if not ad_badge:
            page = i
            name = item.find('div', attrs = {'class' : 'name'}).get_text()
            price = item.find('strong', attrs = {'class' : 'price-value'}).get_text()
            
            rate = item.find('em', attrs = {'class' : 'rating'})
            if rate:
                rate = rate.get_text() 
                
            ## 위의 rate에서 else: continue를 사용했다면, 아래의 내용은 위 rate에 맞춰 들여쓰기를 할 필요가 없다!!
            
            
                rate_count = item.find('span', attrs = {'class' : 'rating-total-count'})
                if rate_count:
                    rate_count = rate_count.get_text()[1:-1]
                link = 'https://www.coupang.com/' + item.find('a', attrs = {'class' : 'search-product-link'})['href']
                if float(rate) >= 4.5 and int(rate_count) >= 100:
                    li_page.append(page)
                    li_name.append(name)
                    li_price.append(price)
                    li_rate.append(rate)
                    li_rate_count.append(rate_count)
                    li_link.append(link)

li_notebook =[li_page,li_name, li_price, li_rate, li_rate_count, li_link]
for i in range(0,6):
    notebook.iloc[:,i] = li_notebook[i]
        
display(notebook)

,page,name,price,rate,rate_count,link
0,1,"LG전자 2020 울트라 PC 14, 화이트, 셀러론, 512GB, 8GB, WIN...","526,000",4.5,415,https://www.coupang.com//vp/products/484154876...
1,1,LG전자 2021 울트라 PC 15.6 + HDMI케이블 + 무선마우스 + 마우스패...,"799,000",5.0,403,https://www.coupang.com//vp/products/467998925...
2,1,"LG전자 2020 울트라 PC 14, 화이트, 셀러론, 64GB, 4GB, WIN1...","379,000",4.5,114,https://www.coupang.com//vp/products/190054140...
3,1,\n ...,"1,598,000",5.0,642,https://www.coupang.com//vp/products/488978269...
4,1,\n ...,"1,448,840",5.0,8160,https://www.coupang.com//vp/products/432248113...
...,...,...,...,...,...,...
77,5,"레노버 2021 ThinkPad E15, 블랙, 라이젠3 4세대, 256GB, 8G...","1,303,140",5.0,408,https://www.coupang.com//vp/products/576213137...
78,5,"삼성전자 2020 갤럭시북 S 13.3, 머큐리 그레이, 코어i3, 256GB, 8...","799,000",5.0,817,https://www.coupang.com//vp/products/231624269...
79,5,"에이서 2021 Swift 3 14, 블루, 라이젠5, 256GB, 8GB, Lin...","748,000",5.0,117,https://www.coupang.com//vp/products/575466085...
80,5,"레노버 2020 V15, 아이언 그레이, 라이젠3 2세대, 256GB, 4GB, F...","379,000",5.0,213,https://www.coupang.com//vp/products/226946415...


### 5) 엑셀파일로 저장

In [336]:
notebook.to_excel('C:/Users/Park/Desktop/data/sample.xlsx')